In [ ]:
import os
import numpy as np
import gc
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
import pickle
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve
from scipy.fftpack import dct

In [ ]:
X = np.load('../SavedFeatures/X_cqcc.npy')
y = np.load('../SavedFeatures/y_cqcc.npy')
pad_height = (3, 0)  
pad_width = (0, 0)   
X = np.pad(X, pad_width=((0, 0), pad_height, pad_width), mode='constant', constant_values=0)
X.shape

(15918, 32, 109)

In [ ]:
X_train, X_val, y_train, y_val= train_test_split(X, y, test_size = 0.2, random_state = 42)

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

del X, y  
gc.collect()

In [29]:
def cus_CNN(input_shape, num_classes):
    model_input = Input(shape=input_shape) 
    x = Conv2D(32, kernel_size=(3, 3), activation='relu')(model_input)
    x = BatchNormalization()(x) 
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.4)(x)  
    
    x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
    x = Dense(128, activation='relu')(x) 
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)  

    x = Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
    x = Dense(64, activation='relu')(x) 
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.2)(x)  
    
    x = Flatten()(x)
    x = Dense(32, activation='relu')(x) 
    x = Dropout(0.1)(x)  
    
    output = Dense(num_classes, activation='softmax')(x) 
    return Model(inputs=model_input, outputs=output)

In [30]:
cnnmodel = cus_CNN(input_shape=(32, 109, 1), num_classes=2)
cnnmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


In [31]:
tf.keras.backend.clear_session()

In [ ]:
cnnmodel.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

In [33]:
test_loss, test_accuracy = cnnmodel.evaluate(X_val, y_val, verbose=0)
y_pred = cnnmodel.predict(X_val)
y_pred_classes = y_pred.argmax(axis=1)
y_true_classes = y_val.argmax(axis=1) 

# Calculate F1-score
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("===================CNN - MFCC===================")
print(f"Accuracy: {test_accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")
eers = []
for i in range(y_pred.shape[1]):
    y_true_binary = y_val[:, i]
    y_pred_prob = y_pred[:, i]
    fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_prob)
    fnr = 1 - tpr
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]
    eers.append(eer)
    print(f"Class {i}: EER = {eer:.5f} at threshold {eer_threshold:.5f}")
# Average EER across classes
mean_eer = np.mean(eers)
print(f"EER: {mean_eer:.5f}")


80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
===================CNN - MFCC===================
Accuracy: 0.93286
F1-Score: 0.93247
Class 0: EER = 0.07143 at threshold 0.85711
Class 1: EER = 0.07213 at threshold 0.14355
EER: 0.07178


In [ ]:
cnnmodel.save("../SavedModels/CQCC_cus_cnn.h5")

In [ ]:

del cnnmodel, X_train, X_test, y_train, y_test  
del test_loss, test_accuracy
del y_pred, y_pred_classes, y_true_classes, f1
del y_true_binary, y_pred_prob, fpr, tpr, thresholds, fnr, eer_threshold, eer
del eers, mean_eer
gc.collect()


1043